In [4]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import numpy as np
from numpy import array
import andi 
import pandas as pd
import matplotlib.pyplot as plt
import keras as kr
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence


In [5]:
import sys
sys.path.append('../')
from src.features import Q_measure_2D, Dist_distribution, Convex_hull
from src.andi_tools import Andi_to_xy, group_by_length, group_similar_as
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Import and prepare the data

In [2]:
all_2D =  pd.read_csv("../data_generation/test_2d.csv").values[:,2:]
list_2D_total = [all_2D[i][~np.isnan(all_2D[i])].reshape(1,-1) for i in np.arange(len(all_2D))]
xdata, ydata = Andi_to_xy(list_2D_total)
output_2d = pd.read_csv("../data_generation/label_2d.csv").values[:,1]
output = np.array([kr.utils.to_categorical(output_2d[i], num_classes=5) for i in range(len(output_2d))])

all_2D = pd.read_csv("../../../ANDI_data/data_2D_task2.csv").values[:,1:]
list_2D_total = [all_2D[i][~np.isnan(all_2D[i])].reshape(1,-1) for i in np.arange(len(all_2D))]
xdata, ydata = Andi_to_xy(list_2D_total)

output_2d = pd.read_csv("../../../ANDI_data/labels_2D_task2.csv").values[:,2]
output = np.array([kr.utils.to_categorical(output_2d[i], num_classes=5) for i in range(len(output_2d))])

In [3]:
output.shape

(500, 5)

# Prepare the training set and calculate the features

In [19]:
# Calculating the different features 

Q_list, lens = Q_measure_2D(xdata, ydata , 2)
Q_list_2, lens_2 = Q_measure_2D(xdata, ydata , 8)
Dist_list, lens_dist = Dist_distribution(xdata, ydata)
Convex_list, conv_dist = Convex_hull(xdata, ydata, 5)

In [20]:
# Padding the Q-values

result = np.squeeze(np.array([np.pad(i,((0,0),(0,np.max(lens)-i.shape[1]))) for i in Q_list]),axis=1)
result_2 = np.squeeze(np.array([np.pad(i,((0,0),(0,np.max(lens_2)-i.shape[1]))) for i in Q_list_2]),axis=1)
result_3 = np.squeeze(np.array([np.pad(i,((0,0),(0,np.max(conv_dist)-i.shape[1]))) for i in Convex_list]),axis=1)
result_d = np.array(Dist_list)

In [21]:
# Splitting the data into a training and test set 

fraction = np.int(len(xdata)*0.75)

result_train, result_test = result[:fraction], result[fraction:]
result2_train, result2_test = result_2[:fraction], result_2[fraction:]
result3_train, result3_test = result_3[:fraction], result_3[fraction:]
resultd_train, resultd_test = result_d[:fraction], result_d[fraction:]

output_train, output_test = output[:fraction], output[fraction:]

# Definition of the model

In [22]:
# define two sets of inputs
inputA = kr.layers.Input(shape=(result_train.shape[1],))
inputB = kr.layers.Input(shape=(result2_train.shape[1],))
inputC = kr.layers.Input(shape=(result3_train.shape[1],))
inputD = kr.layers.Input(shape=(resultd_train.shape[1],))
# Side A 
x1 =  kr.layers.Dense(32, activation="sigmoid")(inputA)
x1 =  kr.layers.Dense(32, activation="sigmoid")(x1)
x1 =  kr.Model(inputs=inputA, outputs=x1)
# Side B
x2 = kr.layers.Dense(32, activation="sigmoid")(inputB)
x2 = kr.layers.Dense(32, activation="sigmoid")(x2)
x2 = kr.Model(inputs=inputB, outputs=x2)
# Side C
x3 =  kr.layers.Dense(32, activation="sigmoid")(inputC)
x3 =  kr.layers.Dense(32, activation="sigmoid")(x3)
x3 =  kr.Model(inputs=inputC, outputs=x3)
# Side D
x4 =  kr.layers.Dense(32, activation="sigmoid")(inputD)
x4 =  kr.layers.Dense(32, activation="sigmoid")(x4)
x4 =  kr.Model(inputs=inputD, outputs=x4)
# Combine side A en B
combined = kr.layers.concatenate([x1.output, x2.output, x3.output, x4.output])
z = kr.layers.Dense(4, activation="sigmoid")(combined)
z = kr.layers.Dense(output_train.shape[1], activation="softmax")(z)

In [23]:
model = kr.Model(inputs=[x1.input, x2.input, x3.input, x4.input], outputs=z)
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=["accuracy"])
#print(model.summary())

In [24]:
output_train.shape

(375, 5)

In [25]:
model.fit([result_train,result2_train,result3_train,resultd_train], output_train, epochs=10, batch_size=10, verbose=1)

Epoch 1/10
375/375 [==============================] - 0s 955us/step - loss: nan - accuracy: 0.2880
Epoch 2/10
375/375 [==============================] - 0s 237us/step - loss: nan - accuracy: 0.2800
Epoch 3/10
375/375 [==============================] - 0s 235us/step - loss: nan - accuracy: 0.2800
Epoch 4/10
375/375 [==============================] - 0s 208us/step - loss: nan - accuracy: 0.2800
Epoch 5/10
375/375 [==============================] - 0s 297us/step - loss: nan - accuracy: 0.2800
Epoch 6/10
375/375 [==============================] - 0s 249us/step - loss: nan - accuracy: 0.2800
Epoch 7/10
375/375 [==============================] - 0s 197us/step - loss: nan - accuracy: 0.2800
Epoch 8/10
375/375 [==============================] - 0s 187us/step - loss: nan - accuracy: 0.2800
Epoch 9/10
375/375 [==============================] - 0s 209us/step - loss: nan - accuracy: 0.2800
Epoch 10/10
375/375 [==============================] - 0s 198us/step - loss: nan - accuracy: 0.2800


In [12]:
model.evaluate([result_test,result2_test,result3_test,resultd_test], output_test)

125/125 [==============================] - 0s 2ms/step


[1.064753496170044, 0.8159999847412109]

In [17]:
model_json = model.to_json()
with open("../saved_models/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("../saved_models/model.h5")
print("Saved model to disk")

Saved model to disk
